<a href="https://colab.research.google.com/github/alex-smith-uwec/NLP_Spring2025/blob/main/FineTune_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#If you plan to get actually get to the fine tuning stage at the bottom of this notebook, then before you begin you should change the runtime to GPU. If you are just noodling around with stuff before that, then just leave things with CPU.

The content of this notebook is adapted from the video below by Lewis Tunstall.


In [1]:
# @title

from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/u--UVvH-LIQ?si=EqRlGOizWG7tgF7b" frameborder="0" allowfullscreen></iframe>')


/usr/local/lib/python3.11/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [2]:
# !pip install 'accelerate>=0.21.0' -U -q

In [3]:
!pip install 'transformers[torch]' -U -q
# !pip install 'transformers[tensorflow]'


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.1 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)


4.51.0


In [5]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [6]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification

import torch
from torch import nn

import random
from sklearn.metrics import f1_score

# The Emotion Dataset

 [The emotion dataset](https://huggingface.co/datasets/dair-ai/emotion)
 dataset card on huggingface

In [7]:
emotion_dataset=load_dataset("emotion",trust_remote_code=True)
emotion_dataset

README.md:   0%|          | 0.00/9.05k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [8]:
random_integer = random.randint(0, 16000)
random_integer

emotion_dataset["train"][random_integer]

{'text': 'i feel as if i was abused in some way', 'label': 0}

In [9]:
emotion_df=emotion_dataset["train"].to_pandas()
emotion_df[10:15]

,text,label
10,i feel like i have to make the suffering i m s...,0
11,i do feel that running is a divine experience ...,1
12,i think it s the easiest time of year to feel ...,3
13,i feel low energy i m just thirsty,0
14,i have immense sympathy with the general point...,1


In [10]:
features=emotion_dataset["train"].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [11]:
features["label"].int2str(3)

'anger'

In [12]:
id2label={idx:features["label"].int2str(idx) for idx in range(6)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [13]:
label2id={v:k for k,v in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

In [14]:
emotion_df["label"].value_counts(normalize=True).sort_index()
#See 5:32 minute mark: distribution of values is very uneven!

,proportion
label,
0,0.291625
1,0.335125
2,0.081500
3,0.134937
4,0.121063
5,0.035750


[sklearn F1 score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

For evaluation of the model, we will use an F-score, which is often used for imbalanced situtations.

#Tokenize everything

#Pretrained model checkpoint [huggingface card](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased)

In [15]:
model_ckpt="microsoft/MiniLM-L12-H384-uncased"
#ckpt is "checkpoint"

tokenizer=AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [16]:
# sample="I did not go running"
sample=emotion_dataset["train"]["text"][0]

encoded_input = tokenizer(sample, return_tensors='pt')

# Print the token IDs (numbers)
print(encoded_input['input_ids'][0])

# Convert the token IDs back to tokens (subwords) and print them
tokens = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
print(tokens)

# Print the original text for reference
print(sample)
encoded_input

tensor([  101,  1045,  2134,  2102,  2514, 26608,   102])
['[CLS]', 'i', 'didn', '##t', 'feel', 'humiliated', '[SEP]']
i didnt feel humiliated


{'input_ids': tensor([[  101,  1045,  2134,  2102,  2514, 26608,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [17]:
def tokenize_text(examples):
  return tokenizer(examples["text"], truncation=True, max_length=512)

In [18]:
emotion_dataset=emotion_dataset.map(tokenize_text, batched=True)
emotion_dataset

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

# Dealing with the imbalanced classes

In [19]:
emotion_df["label"].value_counts()

,count
label,
1,5362
0,4666
3,2159
4,1937
2,1304
5,572


In [20]:

class_weights=(1-(emotion_df["label"].value_counts().sort_index()/len(emotion_df))).values
print(class_weights)

class_weights=torch.from_numpy(class_weights)
print(class_weights)

class_weights=class_weights.float()
print(class_weights)

#.to("cuda") if GPU
class_weights=class_weights#.to("cuda")
print(class_weights)

[0.708375  0.664875  0.9185    0.8650625 0.8789375 0.96425  ]
tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643], dtype=torch.float64)
tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643])
tensor([0.7084, 0.6649, 0.9185, 0.8651, 0.8789, 0.9643])


In [21]:
##See 12:57 minute mark of video
emotion_dataset=emotion_dataset.rename_column("label","labels")

In [22]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [23]:
batch_size = 64
# Assuming `emotion_dataset["train"]` is defined elsewhere and accessible
logging_steps = len(emotion_dataset["train"]) // batch_size
output_dir = "minilm-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    # evaluation_strategy="epoch", (seems to have been replaced in newer versions of transformers with next line)
    eval_steps=logging_steps,
    logging_steps=logging_steps,
    fp16=True,  # Changed to False for CPU compatibility
    push_to_hub=True,  # Set based on your needs
    report_to="none",  # This disables integration with W&B
)


**The class Trainer has a method named compute_loss.
But we are going to define a subclass of Trainer so that we can override compute_loss**

In [24]:
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):  # Add num_items_in_batch
        # Move class_weights to the same device as the model
        device = next(model.parameters()).device  # Get the device of the model
        self.class_weights = class_weights.to(device)  # Move class_weights to the device

        # Feed inputs to model and extract logits
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Extract labels
        labels = inputs.get("labels")
        # Define loss function with class weights
        loss_func = nn.CrossEntropyLoss(weight=self.class_weights)  # Use self.class_weights
        # Compute loss
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [25]:
model=AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                      num_labels=6,
                                                      id2label=id2label,
                                                      label2id=label2id)

pytorch_model.bin:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    accuracy = accuracy_score(labels, preds)  # Calculate accuracy
    return {"f1": f1, "accuracy": accuracy}


In [27]:
trainer=WeightedLossTrainer(model=model,
                          args=training_args,
                          compute_metrics=compute_metrics,
                          train_dataset=emotion_dataset["train"],
                          eval_dataset=emotion_dataset["validation"],
                          tokenizer=tokenizer)

<ipython-input-27-deda7f114c92>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  trainer=WeightedLossTrainer(model=model,


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

In [28]:
trainer.train()

Step,Training Loss
250,1.414500
500,1.012900
750,0.698500
1000,0.514800
1250,0.445200


TrainOutput(global_step=1250, training_loss=0.8171847595214844, metrics={'train_runtime': 109.9288, 'train_samples_per_second': 727.744, 'train_steps_per_second': 11.371, 'total_flos': 581870908894464.0, 'train_loss': 0.8171847595214844, 'epoch': 5.0})

In [29]:
test_results = trainer.evaluate(emotion_dataset["test"])
print(test_results)


{'eval_loss': 0.39904963970184326, 'eval_f1': 0.9000251497303563, 'eval_accuracy': 0.9035, 'eval_runtime': 4.3523, 'eval_samples_per_second': 459.532, 'eval_steps_per_second': 57.441, 'epoch': 5.0}


# Inference

In [30]:
from transformers import pipeline

In [31]:
model_ckpt="alex-smith/minilm-finetuned-emotion"

In [32]:
pipe=pipeline("text-classification",model=model_ckpt)

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


In [33]:
pipe("I really dislike integration by parts if we have to do it twice!")

[{'label': 'sadness', 'score': 0.4801639914512634}]

In [34]:

# Select a sample text from the training split
train_sample=random.randint(0, 16000)
test_sample=random.randint(0,2000)

train_text = emotion_dataset["train"]["text"][train_sample]
train_label=emotion_dataset["train"]["labels"][train_sample]

# Select a sample text from the testing split
test_text = emotion_dataset["test"]["text"][test_sample]
test_label=emotion_dataset["test"]["labels"][test_sample]

# Classify the training text
train_result = pipe(train_text)
print(f"Training text classification result:\n {train_result}, actual label: {id2label[train_label]}")
print(f"train text: {train_text}")

print(f"\n")
# Classify the testing text
test_result = pipe(test_text)
print(f"Testing text classification result:\n {test_result}, actual label: {id2label[test_label]}")
print(f"test text: {test_text}")

Training text classification result:
 [{'label': 'sadness', 'score': 0.8837149143218994}], actual label: sadness
train text: i do find new friends i m going to try extra hard to make them stay and if i decide that i don t want to feel hurt again and just ride out the last year of school on my own i m going to have to try extra hard not to care what people think of me being a loner


Testing text classification result:
 [{'label': 'love', 'score': 0.6138788461685181}], actual label: love
test text: i smiled at him feeling his longing and said maybe later buddy but i have to make lunch now
